<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#예시-1)" data-toc-modified-id="예시-1)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>예시 1)</a></span><ul class="toc-item"><li><span><a href="#return-calculation" data-toc-modified-id="return-calculation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>return calculation</a></span><ul class="toc-item"><li><span><a href="#&quot;forward&quot;-수익률-계산" data-toc-modified-id="&quot;forward&quot;-수익률-계산-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>"forward" 수익률 계산</a></span></li><li><span><a href="#수익률-계산" data-toc-modified-id="수익률-계산-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>수익률 계산</a></span></li><li><span><a href="#freq='D'" data-toc-modified-id="freq='D'-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>freq='D'</a></span></li><li><span><a href="#freq='B'" data-toc-modified-id="freq='B'-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>freq='B'</a></span></li><li><span><a href="#freq-with-MultiIndex" data-toc-modified-id="freq-with-MultiIndex-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>freq with MultiIndex</a></span></li></ul></li></ul></li><li><span><a href="#예시-2)" data-toc-modified-id="예시-2)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>예시 2)</a></span><ul class="toc-item"><li><span><a href="#DataFrame.index" data-toc-modified-id="DataFrame.index-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DataFrame.index</a></span></li><li><span><a href="#inplace" data-toc-modified-id="inplace-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>inplace</a></span></li></ul></li><li><span><a href="#DataFrame.set_index" data-toc-modified-id="DataFrame.set_index-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DataFrame.set_index</a></span><ul class="toc-item"><li><span><a href="#sort_values" data-toc-modified-id="sort_values-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>sort_values</a></span></li></ul></li></ul></div>

asfreq(): Convert TimeSeries to specified frequency
- month-end, year-end, business month-end, business month-start 등. 
    - df.asfreq('M'), df.asfreq('A'), df.asfreq('BM'), df.asfreq('BMS') (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)



# 예시 1)
- groupby, shift method의 작동방식을 익히기 위한 간단한 수익률 계산 예시
- np.random.normal(loc=loc, scale=scale, size=size)
    - 평균이 loc, 표준편차가 scale, 크기가 size인 정규분포난수

In [2]:
import numpy as np
import pandas as pd
df = pd.read_excel(r"E:\Dropbox\GitHub\MFE\sample_dsf.xlsx", engine='openpyxl')
print(df.columns.tolist())

df = df[['DATE', 'PERMNO', 'PRC', 'RET']] # keep only needed
display(df)

['CUSIP', 'PERMNO', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'DATE', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD', 'RETX']


,DATE,PERMNO,PRC,RET
0,2019-01-02,10026,141.000000,-0.024829
1,2019-01-03,10026,143.020004,0.014326
2,2019-01-04,10026,144.839996,0.012725
3,2019-01-07,10026,145.410004,0.003935
4,2019-01-08,10026,148.699997,0.022626
...,...,...,...,...
155,2019-02-22,10044,9.100000,-0.007417
156,2019-02-25,10044,9.250000,0.016483
157,2019-02-26,10044,9.450000,0.021622
158,2019-02-27,10044,9.356800,-0.009862


## return calculation
- pct_change(periods, freq)를 통해 계산 가능: ( price(t) - price(t-periods) ) / price(t-periods)


    - pct_change(periods=1): daily frequency 데이터의 경우 1일 수익률을, monthly frequency 데이터의 경우 1달 수익률을 계산
    - pct_change(periods=-1): ( price(t) - price(t+1) ) / price(t+1); 수익률이 아님을 주의
        - "forward" 수익률을 구하기 위해서는 shift() 및 diff()의 조합을 사용    


    - pct_change(periods=1, freq='D'): index의 type이 datetime인 경우, 수익률을 계산
        - freq='D'는 calendar day 기준이므로, 날짜 사이에 휴일/주말이 낀 경우 수익률을 계산하지 않음
        - freq='B'는 business day 기준으로, 일반적으로 사용하는 수익률


### "forward" 수익률 계산

In [3]:
_df = df.groupby(by=['PERMNO'])['PRC']
print( ((_df.shift(periods=-1) - _df.shift(periods=0)) / _df.shift(periods=0)).head()) # method 1
print('')
print( (-df.groupby(by=['PERMNO'])['PRC'].diff(periods=-1) / df.groupby(by=['PERMNO'])['PRC'].shift(periods=0)).head()) # method 2

0    0.014326
1    0.012725
2    0.003935
3    0.022626
4   -0.006658
Name: PRC, dtype: float64

0    0.014326
1    0.012725
2    0.003935
3    0.022626
4   -0.006658
Name: PRC, dtype: float64


### 수익률 계산

In [4]:
print(df.index)
print(df.groupby(by=['PERMNO'])['PRC'].pct_change(periods=1).head())

RangeIndex(start=0, stop=160, step=1)
0         NaN
1    0.014326
2    0.012725
3    0.003935
4    0.022626
Name: PRC, dtype: float64


### freq='D'

In [5]:
df.set_index('DATE', inplace=True)
print(df.index)
print(df.groupby(by=['PERMNO'])['PRC'].pct_change(periods=1, freq='D').head()) # freq='D'

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07',
               '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11',
               '2019-01-14', '2019-01-15',
               ...
               '2019-02-14', '2019-02-15', '2019-02-19', '2019-02-20',
               '2019-02-21', '2019-02-22', '2019-02-25', '2019-02-26',
               '2019-02-27', '2019-02-28'],
              dtype='datetime64[ns]', name='DATE', length=160, freq=None)
DATE
2019-01-02         NaN
2019-01-03    0.014326
2019-01-04    0.012725
2019-01-07         NaN
2019-01-08    0.022626
Name: PRC, dtype: float64


### freq='B'

In [6]:
print(df.groupby(by=['PERMNO'])['PRC'].pct_change(periods=1, freq='B').head()) # freq='D'

DATE
2019-01-02         NaN
2019-01-03    0.014326
2019-01-04    0.012725
2019-01-07    0.003935
2019-01-08    0.022626
Name: PRC, dtype: float64


### freq with MultiIndex
- freq는 index가 MultiIndex가 아닐 때만 작용함 (MultiIndex가 아닌, DatetimeIndex일 경우에만)


In [7]:
df.set_index(['PERMNO', df.index], inplace=True)
print(df.index)

MultiIndex([(10026, '2019-01-02'),
            (10026, '2019-01-03'),
            (10026, '2019-01-04'),
            (10026, '2019-01-07'),
            (10026, '2019-01-08'),
            (10026, '2019-01-09'),
            (10026, '2019-01-10'),
            (10026, '2019-01-11'),
            (10026, '2019-01-14'),
            (10026, '2019-01-15'),
            ...
            (10044, '2019-02-14'),
            (10044, '2019-02-15'),
            (10044, '2019-02-19'),
            (10044, '2019-02-20'),
            (10044, '2019-02-21'),
            (10044, '2019-02-22'),
            (10044, '2019-02-25'),
            (10044, '2019-02-26'),
            (10044, '2019-02-27'),
            (10044, '2019-02-28')],
           names=['PERMNO', 'DATE'], length=160)


In [8]:
df.groupby(by='PERMNO')['PRC'].pct_change(periods=1) # w/o freq works fine with MultiIndex

PERMNO  DATE      
10026   2019-01-02         NaN
        2019-01-03    0.014326
        2019-01-04    0.012725
        2019-01-07    0.003935
        2019-01-08    0.022626
                        ...   
10044   2019-02-22   -0.007417
        2019-02-25    0.016483
        2019-02-26    0.021622
        2019-02-27   -0.009862
        2019-02-28    0.013167
Name: PRC, Length: 160, dtype: float64

In [9]:
df.groupby(by='PERMNO', level='DATE')['PRC'].pct_change(periods=1) # w/o freq works fine with MultiIndex

PERMNO  DATE      
10026   2019-01-02         NaN
        2019-01-03    0.014326
        2019-01-04    0.012725
        2019-01-07    0.003935
        2019-01-08    0.022626
                        ...   
10044   2019-02-22   -0.007417
        2019-02-25    0.016483
        2019-02-26    0.021622
        2019-02-27   -0.009862
        2019-02-28    0.013167
Name: PRC, Length: 160, dtype: float64

In [10]:
myFun = lambda x: x.pct_change()
print(df.groupby(by=['PERMNO'], level=[1]).apply(myFun)['PRC'].head())

PERMNO  DATE      
10026   2019-01-02         NaN
        2019-01-03    0.014326
        2019-01-04    0.012725
        2019-01-07    0.003935
        2019-01-08    0.022626
Name: PRC, dtype: float64


In [11]:
df.groupby(by='PERMNO', level='DATE')['PRC'].pct_change(periods=1, freq='B') # freq does not work with MultiIndex

NotImplementedError: Not supported for type MultiIndex

# 예시 2)
- DataFrame.index, rename_axis, inplace 등의 사용 예시
- pandas에서 "y" 변수가 하나일 때는 pandas.Series, 2개 이상일 때는 pandas.DataFrame으로 생성되며, 각각에 대한 attribute/method가 다름을 주의

- pandas.Series는 to_frame() method를 통해 pandas.DataFrame으로 변환 가능
    - 예) y = pd.Series([1, 3, 5]).to_frame(); print(y); print(type(y))
    

- 일반적으로는 pandas.Series를 가능한 한 사용하지 않고 pandas.DataFrame을 사용하는 것을 추천

In [29]:
import numpy as np
import pandas as pd

dates = pd.date_range('2015-01-01', periods=7, freq='W-Thu') # 매주 목요일에만 날짜 생성
index = ['005930 KS Equity', '005930 KS Equity', '005380 KS Equity', '005380 KS Equity', 
         '005930 KS Equity', 'SPX US Index', 'SPX US Index']

data1 = np.ones_like(dates, dtype='float') # np.ones_like(dates) not work as expected
price = 100 * np.exp(np.random.normal(loc=0, scale=1, size=len(dates)))

df1 = pd.DataFrame(data=data1, columns=['var1'], index=index)
df2 = pd.DataFrame(data=price, columns=['price'], index=[index, dates])

In [30]:
display(df1, df2)

,var1
005930 KS Equity,1.0
005930 KS Equity,1.0
005380 KS Equity,1.0
005380 KS Equity,1.0
005930 KS Equity,1.0
SPX US Index,1.0
SPX US Index,1.0


price
005930 KS Equity 2015-01-01  102.686917
                 2015-01-08  167.289767
005380 KS Equity 2015-01-15  115.485388
                 2015-01-22   56.074052
005930 KS Equity 2015-01-29   57.806519
SPX US Index     2015-02-05   26.320606
                 2015-02-12   65.600643

## DataFrame.index
- DataFrame.Index.name 을 통해 index의 naming이 가능
- 보다 일반적으로는 DataFrame.Index.set_names()를 통해 index의 naming이 가능


- rename_axis()를 통해 index의 re-naming이 가능
    - 기존 name이 없을 시, rename_axis()로는 naming 불가

In [31]:
print(f'df1.index.name: {df1.index.name}')

print('\n')
df1.index.name = 'abcde'
print(f'df1.index.name: {df1.index.name}')

print('\n')
df1.index.set_names(['secid'])
print(f'df1.index.name: {df1.index.name}')

df1.index.name: None


df1.index.name: abcde


df1.index.name: abcde


In [32]:
df1.index.name = None
print(f'df1.index.name: {df1.index.name}')
df1.rename_axis(index=dict(secid='idx'), axis=0, inplace=True)
print(f'df1.index.name: {df1.index.name}') # rename_axis() not work
print('\n')
df1.index.name = 'abcd'
print(f'df1.index.name: {df1.index.name}')
df1.rename_axis(index=dict(abcd='idx'), axis=0, inplace=True)
print(f'df1.index.name: {df1.index.name}') # rename_axis() works

df1.index.name: None
df1.index.name: None


df1.index.name: abcd
df1.index.name: idx


## inplace
- DataFrame의 method들의 경우, inplace 옵션이 존재하는 경우가 있음
- inplace 옵션의 default는 False이며, 이 경우 method의 결과인 object를 반환
- 기존 object에 결과를 할당하지 않는 한, 결과값이 update되지 않음

In [33]:
print(df1.rename_axis(index=dict(idx='secid'), axis=0, inplace=False)) # index이름이 'secid'인 DataFrame을 반환
print(df1.index.names) # 'secid'로 update되지 않은 'idx'를 반환

df1 = df1.rename_axis(index=dict(idx='secid'), axis=0) # 결과값을 원래 DataFrame에 재할당
print(df1)
print(df1.index.names) # index 이름이 'secid'임을 확인 가능

                  var1
secid                 
005930 KS Equity   1.0
005930 KS Equity   1.0
005380 KS Equity   1.0
005380 KS Equity   1.0
005930 KS Equity   1.0
SPX US Index       1.0
SPX US Index       1.0
['idx']
                  var1
secid                 
005930 KS Equity   1.0
005930 KS Equity   1.0
005380 KS Equity   1.0
005380 KS Equity   1.0
005930 KS Equity   1.0
SPX US Index       1.0
SPX US Index       1.0
['secid']


# DataFrame.set_index
- set_index()를 통해, index를 재정의 가능
- index를 교체하거나, MultiIndex로 바꾸는 경우
    - MultiIndex는 (firm name, date) 등으로 사용 가능
    - 옵션의 경우, (underlying name, option name, date, maturity, strike_price, call-put flag) 등을 사용 가능
    - 예) (SPX US Equity, SPX US 01/15/21 C3500 Index, '2021-01-04', '2021-01-15', 3500, 'Call')

In [34]:
df1.set_index([df1.index, dates], inplace=True) # index -> MultiIndex
print(df1.index)

df1.rename_axis(index={None:'date'}, axis=0, inplace=True)
print(df1.index.names)

MultiIndex([('005930 KS Equity', '2015-01-01'),
            ('005930 KS Equity', '2015-01-08'),
            ('005380 KS Equity', '2015-01-15'),
            ('005380 KS Equity', '2015-01-22'),
            ('005930 KS Equity', '2015-01-29'),
            (    'SPX US Index', '2015-02-05'),
            (    'SPX US Index', '2015-02-12')],
           names=['secid', None])
['secid', 'date']


In [35]:
df1.index.set_names(['secid', 'date'], inplace=True)
df2.index.set_names(['secid', 'date'], inplace=True)
display(df1, df2)

var1
secid            date            
005930 KS Equity 2015-01-01   1.0
                 2015-01-08   1.0
005380 KS Equity 2015-01-15   1.0
                 2015-01-22   1.0
005930 KS Equity 2015-01-29   1.0
SPX US Index     2015-02-05   1.0
                 2015-02-12   1.0

price
secid            date                  
005930 KS Equity 2015-01-01  102.686917
                 2015-01-08  167.289767
005380 KS Equity 2015-01-15  115.485388
                 2015-01-22   56.074052
005930 KS Equity 2015-01-29   57.806519
SPX US Index     2015-02-05   26.320606
                 2015-02-12   65.600643

## sort_values
- index및 column에 대해 (row-wise) sort가 가능
- ascending은 모든 변수에 대해 True가 default
    - ascending order는 변수 길이에 해당하는 Boolean list를 통해 변경 가능

In [45]:
df1.sort_values(by=['secid', 'date'], ascending=[True, False], inplace=True) # sort by index
df2.sort_values(by=['price'], inplace=True) # sort by column

In [46]:
display(df1, df2)

var1
secid            date            
005380 KS Equity 2015-01-22   1.0
                 2015-01-15   1.0
005930 KS Equity 2015-01-29   1.0
                 2015-01-08   1.0
                 2015-01-01   1.0
SPX US Index     2015-02-12   1.0
                 2015-02-05   1.0

,,price
secid,date,
SPX US Index,2015-02-05,26.320606
005380 KS Equity,2015-01-22,56.074052
005930 KS Equity,2015-01-29,57.806519
SPX US Index,2015-02-12,65.600643
005930 KS Equity,2015-01-01,102.686917
005380 KS Equity,2015-01-15,115.485388
005930 KS Equity,2015-01-08,167.289767
